# Imports and Preprocessing

In [92]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

In [93]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

# ANN Model

In [94]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal

# Define function to create the ANN model 
def create_model(n_hidden, n_neurons, learning_rate, activation, input_shape=[3]): 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Define function to build the model
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3 , activation = 'relu'):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate, activation=activation) 

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30, 40, 50],
    "activation": ['relu', 'sigmoid', 'tanh']
}

# Create the model and optimize with Randomized Search
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Fit the model on the training data
rnd_search_cv_ANN.fit(data_train, target_train)
print(rnd_search_cv_ANN.best_params_)

# Predict the target values
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)

mse = mean_squared_error(target_test, target_pred_ANN)
print("MSE:", mse)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'tanh', 'batch_size': 16, 'epochs': 30, 'learning_rate': 0.000433062956670166, 'n_hidden': 3, 'n_neurons': 52}
MSE: 127116.67691456934


# DT using LightGBM

In [95]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create the model and define the hyperparameters to search for the Randomized Search
model_DT = lgb.LGBMRegressor()
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the Randomized Search object
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3, scoring='neg_mean_squared_error')

# Fit the model on the training data
rnd_search_cv_DT.fit(data_train, target_train)

# Get the best parameters and best estimator
best_params = rnd_search_cv_DT.best_params_
best_estimator = rnd_search_cv_DT.best_estimator_

# Make predictions
target_pred_DT = best_estimator.predict(data_test)

# Calculate the MSE
mse = mean_squared_error(target_test, target_pred_DT)
print("MSE:", mse)



[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000034 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49
[LightGBM] [Info] Number of data points in the train set: 138, number of used features: 3
[LightGBM] [Info] Start training from score 1561.014493
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# Predicting Order Quantities 

In [96]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((c - s) + (p - c))
print(critical_ratio)

0.5


In [97]:
from scipy.stats import norm

# Calculate the forecast error for ANN
target_test_flatten = target_test.values.flatten()
target_pred_ANN_flatten = target_pred_ANN.flatten()
forecast_error_ANN = target_test_flatten - target_pred_ANN_flatten

# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_ANN.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_ANN = np.array(final_order_quantities_ANN)

In [98]:
from scipy.stats import norm

# Calculate the forecast error for ANN
target_test_flatten = target_test.values.flatten()
target_pred_DT_flatten = target_pred_DT.flatten()
forecast_error_DT = target_test_flatten - target_pred_DT_flatten

# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_DT[i] + np.random.choice(forecast_error_DT, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_DT.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_DT = np.array(final_order_quantities_DT)

In [99]:
print(final_order_quantities_ANN)

[1544.99407227 1528.15756018 1540.79974988 1547.39996741 1556.99623682
 1549.37255872 1531.68974756 1536.2431886  1546.80061438 1539.20313342
 1533.39238367 1513.87191846 1565.42040356 1534.42520056 1568.59689771
 1568.26206921 1541.51832129 1551.84315125 1543.64453625 1552.77198438
 1544.9399375  1534.54737842 1542.92746411 1555.75669824 1520.03229919
 1546.73367578 1537.46578906 1543.82056042 1553.58521069 1558.1343833
 1534.02227856 1562.43066589 1550.10849597 1534.63453345 1554.42064124
 1540.41366113 1535.26644446 1546.92952148 1550.17012708 1550.94757349
 1525.04477393 1540.80550635 1566.66692175 1557.91648511 1539.76710608
 1560.30855615 1549.44425513 1543.93252368 1556.65111267 1567.92836255
 1528.82431238 1555.5806084  1538.37299353]


# Calculate Costs

In [103]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
overall_costs_ANN = 0
overall_costs_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * round(final_order_quantities_ANN[i])
    profit_DT = (p - c) * round(final_order_quantities_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if final_order_quantities_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - round(final_order_quantities_ANN[i]))
    if final_order_quantities_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (round(final_order_quantities_ANN[i]) - target_test.values[i])
    if final_order_quantities_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - round(final_order_quantities_DT[i]))
    if final_order_quantities_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (round(final_order_quantities_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT

    # Calculate the total costs for the week
    total_costs_ANN = overstock_ANN + understock_ANN
    overall_costs_ANN += total_costs_ANN
    total_costs_DT = overstock_DT + understock_DT
    overall_costs_DT += total_costs_DT
   

# Print the overall profit
print('Overall profit for ANN: ', overall_profit_ANN)
print('Overall costs for ANN: ', overall_costs_ANN)
print('Overall profit for DT: ', overall_profit_DT)
print('Overall costs for DT: ', overall_costs_DT)

Overall profit for ANN:  [66371]
Overall costs for ANN:  [15553]
Overall profit for DT:  [77151]
Overall costs for DT:  [4842]


In [101]:
possible_profit = (p - c) * target_test.values.sum()
print('Possible profit: ', possible_profit)

Possible profit:  81991
